# CONSTANTS USED FOR GENERATION. FEEL FREE TO EDIT.
To edit outputs, scroll down until you find the next massive header.

In [ ]:
# What types of areas different rooms can be
AREA_TYPES = ["desk", "room", "building"]
# The types of features rooms can include
FEATURES = ["Screen", "Camera", "Microphone"]

# How many users to generate
USER_COUNT = 500
# How many areas to generate
AREA_COUNT = 20
# For how long into the future bookings should be generated.
# Note that this is PER AREA, so making both
# `BOOKING_DURATION_PER_AREA` and `AREA_COUNT`
# high is strongly advised against.
BOOKING_DURATION_PER_AREA = 60 * 24 * 7


***
Code below
***

In [ ]:
import random
import string
from datetime import datetime, time, date, timedelta

def rand_text(length: int=None, whitespace: bool = False) -> str:
	text = string.ascii_letters
	if whitespace:
		text = text + " "
	if (not length):
		length = random.randint(3, 20)
	text = "".join(random.choices(text, k=length)).capitalize();
	return text

def rand_bool(odds=0.5) -> bool:
	return random.random() < odds

def choices_x(arr: list, num: int = 3) -> list:
	retval = []
	for _ in range(num):
		idx = random.randint(0, len(arr) - 1)
		retval.append(arr.pop(idx))
	return retval

In [ ]:
class IdTicker:
	index: int = 0;

	def __init__(self):
		self.index = 0

	def get(self) -> int:
		self.index += 1
		return self.index - 1

In [ ]:
user_ticker = IdTicker()
class User:
	id: int = 0
	first_name: str = ""
	last_name: str = ""
	email: str = ""

	def __init__(self):
		self.id = user_ticker.get()
		self.first_name = rand_text()
		self.last_name = rand_text()
		self.email = (rand_text() + "@" + rand_text() + "." + rand_text(random.randint(2,3)))

	def __str__(self):
		return f"{self.first_name} {self.last_name}: {self.email}"

users: dict[int, User] = {}
for i in range(USER_COUNT):
	user = User()
	users[user.id] = user

print(users[1])

In [ ]:
area_ticker = IdTicker()
class Area:
	id: int = 0
	administrators: list[int] = []
	super_area: int = 0
	area_type: str = "desk"
	capacity: int = 0
	name: str = ""
	description: str = ""
	reservable: bool = False;
	features: list[str] = []

	def __init__(self):
		self.id = area_ticker.get()
		self.administrators = choices_x(list(users.values()))
		if (rand_bool(0.2)):
			self.super_area = random.randint(0, AREA_COUNT - 1)
		self.area_type = random.choice(AREA_TYPES)
		self.capacity = random.randint(0, 10) if rand_bool(0.8) else random.randint(10,100)
		self.name = rand_text()
		self.description = rand_text(random.randint(20, 100), whitespace=True)
		self.reservable = rand_bool(0.9)
		self.features = random.choices(FEATURES, k=random.randint(0, len(FEATURES) - 1))

areas = {}
for _ in range(AREA_COUNT):
	area = Area()
	areas[area.id] = area

In [ ]:
booking_ticker = IdTicker()
class Booking:
	id: int = 0
	area: int = 0
	user: int = 0
	start_time: datetime = datetime.now()
	end_time: datetime = datetime.now()
	comment: str = ""

	def __init__(self, area, start, duration):
		self.id = booking_ticker.get()
		self.area = area
		self.user = random.randint(0, USER_COUNT - 1)
		self.start_time = start
		self.end_time = self.start_time + timedelta(minutes=duration)
		self.comment = rand_text(random.randint(20, 100), whitespace=True)

	def __str__(self):
		return f"{self.area}: {self.start_time.time()} - {self.end_time.time()}"

bookings = {}
current_time = datetime.now()
current_time -= timedelta(
	seconds=current_time.second,
	microseconds=current_time.microsecond)
for area in areas:
	current_delta = random.randint(5, 60 * 4)
	while current_delta < BOOKING_DURATION_PER_AREA:
		duration = random.randint(5, 60 * 4)
		start = current_time + timedelta(minutes=current_delta)

		booking = Booking(area, start, duration)
		bookings[booking.id] = booking

		current_delta = current_delta + duration


In [ ]:
import pandas as pd
def dict_to_df(dict):
	return pd.DataFrame([vars(f) for f in list(dict.values())])

In [ ]:
users_pd = dict_to_df(users)
areas_pd = dict_to_df(areas)
bookings_pd = dict_to_df(bookings)

# EDIT THE BELOW VALUES TO CHANGE OUTPUT FORMAT

In [ ]:
import os

# Directory for output
OUTPUT_DIR = "./data/"
# File extension. This does not change the format
OUTPUT_EXTENSION = "json"

# Edit which function is used in order to change format
# For example, for csv, edit to `to_csv`.
# MAKE SURE TO NOT INCLUDE PARANTHESES
OUTPUTS = {
	"users": users_pd.to_json,
	"areas": areas_pd.to_json,
	"bookings": bookings_pd.to_json
}

# None of the code below needs to be touched
os.makedirs(OUTPUT_DIR, exist_ok=True)

for name, output_func in OUTPUTS.items():
	with open(f"{OUTPUT_DIR}{name}.{OUTPUT_EXTENSION}", "w") as file:
		file.write(output_func())

## Populate users

In [ ]:
import random
import requests

# First names
first_names = [
    "Odd", "Børre", "Kjell", "Sverre", "Åsmund", "Einar", "Torbjørn", "Trygve", "Frode", "Alf",
    "Ragnhild", "Synnøve", "Tordis", "Ingvild", "Gudrun", "Solveig", "Liv", "Jorunn", "Gerd", "Grethe"
]

# Last names
last_names = [
    "Haugen", "Nilsen", "Haugland", "Bjørnstad", "Aas", "Solberg", "Myklebust", "Skaug", "Lunde", "Knutsen",
    "Ødegård", "Karlsen", "Håland", "Engen", "Sæther", "Tangen", "Viken", "Storli", "Moen", "Rødseth"
]

def generate_users(n=10):
    users = []
    for _ in range(n):
        first = random.choice(first_names)
        last = random.choice(last_names)
        email = f"{first.lower()}@{last.lower()}.no".replace("ø", "oe").replace("å", "aa").replace("æ", "ae")
        users.append({
            "firstName": first,
            "lastName": last,
            "email": email
        })
    return users

# Create users
users = generate_users(100)

# Optional: POST to backend
def post_users(users):
    url = 'http://localhost:8080/user'
    for user in users:
        try:
            logger.info(f"Posting user: {user['email']}")
            response = requests.post(url, json=user)
            if response.status_code in [200, 201]:
                logger.info("Success")
            else:
                logger.warning(f"Failed: {response.status_code} - {response.text}")
        except requests.RequestException as e:
            logger.error(f"Error: {e}")

# Uncomment to send to backend:
post_users(users)

## Create Area features and types

In [ ]:
import requests

# Area Types
area_types = [
    {'name': 'Desk', 'description': 'A single workstation typically used by one person.'},
    {'name': 'Room', 'description': 'An enclosed space within a building used for a specific purpose.'},
    {'name': 'Building', 'description': 'A standalone structure that houses multiple rooms or areas.'},
    {'name': 'Floor', 'description': 'A level within a building containing multiple rooms or spaces.'},
    {'name': 'Cubicle', 'description': 'A small, semi-enclosed office workspace.'},
    {'name': 'Open Area', 'description': 'A shared, open-concept workspace without physical barriers.'},
    {'name': 'Conference Room', 'description': 'A room designated for meetings and group discussions.'},
    {'name': 'Lobby', 'description': 'The entrance or reception area of a building.'},
    {'name': 'Lounge', 'description': 'A relaxed, informal area for breaks or social interaction.'},
    {'name': 'Storage Room', 'description': 'A space designated for storing supplies or equipment.'}
]

# Feature Keywords
feature_keywords = [
    'Whiteboard', 'Projector', 'Air Conditioning', 'Power Outlets', 'Natural Light',
    'Wi-Fi', 'Soundproofing', 'Smart Board', 'Adjustable Lighting', 'Video Conferencing'
]

def generate_feature_data(keywords):
    return [
        {
            'name': keyword,
            'description': f'This area includes {keyword.lower()} for enhanced functionality.'
        }
        for keyword in keywords
    ]

area_features = generate_feature_data(feature_keywords)

# Base URL for local backend
BASE_URL = 'http://localhost:8080'

def post_data(endpoint, data_list):
    for item in data_list:
        try:
            response = requests.post(f"{BASE_URL}/{endpoint}", json=item)
        except requests.RequestException as e:
            print(f"Error posting {endpoint}: {e}")

# Post area types
post_data('areatype', area_types)

# Post area features
post_data('areafeature', area_features)




## Create areas

In [ ]:
import requests
import random

# Base URL
BASE_URL = 'http://localhost:8080'

# Keyword lists
name_keywords = ["North", "Quiet", "Zone", "Main", "Focus", "Lounge", "Deep", "Creative", "Huddle", "Bay"]
desc_keywords = ["This", "area", "is", "meant", "for", "focused", "work", "collaboration", "with", "natural", "light", "privacy", "elements", "and", "more"]

def get_ids(endpoint):
    try:
        response = requests.get(f'{BASE_URL}/{endpoint}')
        response.raise_for_status()
        return [item['id'] for item in response.json()]
    except Exception as e:
        print(f"Error fetching {endpoint} IDs: {e}")
        return []

def generate_random_area(user_ids, area_type_ids, feature_ids, super_area_ids=None):
    area = {
        "administratorIds": random.sample(user_ids, k=random.randint(1, min(3, len(user_ids)))),
        "superArea": None,
        "areaTypeIds": random.choice(area_type_ids),
        "areaFeatureIds": random.sample(feature_ids, k=random.randint(1, min(5, len(feature_ids)))),
        "capacity": random.randint(0, 50),
        "calendarLink": None,
        "name": " ".join(random.sample(name_keywords, 3)),
        "description": " ".join(random.choices(desc_keywords, k=10)),
        "reservable": random.choices([True, False], weights=[3, 1])[0]
    }

    if super_area_ids and len(super_area_ids) > 0 and random.random() < 0.33:
        area["superArea"] = random.choice(super_area_ids)

    return area

def post_area(area_data):
    try:
        response = requests.post(f'{BASE_URL}/area', json=area_data)
        if response.status_code in [200, 201]:
            print(f"Posted area: {response.json()}")
        else:
            print(f"Failed to post area: {response.status_code} - {response.text}")
    except requests.RequestException as e:
        print(f"Error posting area: {e}")

# Generate & post areas
def generate_and_post_areas(n=5):
    user_ids = get_ids('user')
    area_type_ids = get_ids('areatype')
    feature_ids = get_ids('areafeature')
    existing_area_ids = get_ids('area')  # Does not work due to no /areas endpoint

    if not (user_ids and area_type_ids and feature_ids):
        print("Missing required data to generate areas.")
        return

    for _ in range(n):
        area = generate_random_area(user_ids, area_type_ids, feature_ids, super_area_ids=existing_area_ids)
        post_area(area)

generate_and_post_areas(1000)
